In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelEncoder, OneHotEncoder

pd.set_option('display.max_rows', None)

train = pd.read_csv('/content/drive/MyDrive/물류예측/train_df.csv', encoding="cp949")
test = pd.read_csv('/content/drive/MyDrive/물류예측/test_df.csv', encoding="cp949")

submission = pd.read_csv('/content/drive/MyDrive/물류예측/sample_submission.csv', encoding="cp949")

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
train.shape, test.shape

((32000, 6), (4640, 5))

In [35]:
df = pd.concat([train, test], axis=0)

In [ ]:
df.shape

(36640, 6)

In [36]:
df.columns = ['index', 'send_num', 'receive_num', "cate_big", "cate_mid","target"]

In [ ]:
df.head()

,index,send_num,receive_num,cate_big,cate_mid,target
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0


In [37]:
df['send_num'].nunique(), df['receive_num'].nunique()

(3983, 31413)

In [38]:
df['send_num_3'] = df["send_num"].astype('str').str[:3].astype('int64')
df['send_num_4'] = df['send_num'].astype('str').str[:4].astype('int64')
df['send_num_5'] = df['send_num'].astype('str').str[:5].astype('int64')
df['send_num_6'] = df['send_num'].astype('str').str[:6].astype('int64')
df['send_num_7'] = df['send_num'].astype('str').str[:7].astype('int64')
df['send_num_8'] = df['send_num'].astype('str').str[:8].astype('int64')
df['send_num_9'] = df['send_num'].astype('str').str[:9].astype('int64')
df['send_num_10'] = df['send_num'].astype('str').str[:10].astype('int64')

In [39]:
df['rec_num_3'] = df["receive_num"].astype('str').str[:3].astype('int64')
df['rec_num_4'] = df['receive_num'].astype('str').str[:4].astype('int64')
df['rec_num_5'] = df['receive_num'].astype('str').str[:5].astype('int64')
df['rec_num_6'] = df['receive_num'].astype('str').str[:6].astype('int64')
df['rec_num_7'] = df['receive_num'].astype('str').str[:7].astype('int64')
df['rec_num_8'] = df['receive_num'].astype('str').str[:8].astype('int64')
df['rec_num_9'] = df['receive_num'].astype('str').str[:9].astype('int64')
df['rec_num_10'] = df['receive_num'].astype('str').str[:10].astype('int64')

In [9]:
df.head()

,index,send_num,receive_num,cate_big,cate_mid,target,send_num_3,send_num_4,send_num_5,send_num_6,send_num_7,send_num_8,send_num_9,send_num_10,rec_num_3,rec_num_4,rec_num_5,rec_num_6,rec_num_7,rec_num_8,rec_num_9,rec_num_10
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265
3,3,1154500002014200,5011000315087400,식품,농산물,10.0,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177


In [ ]:
# df.groupby('send_num_4').sum()

# np.log(train.groupby('send_sum_4').sum()['target']).hist()

In [10]:
# 같은 곳에서 같은 곳으로 보내는

same_idx = df.loc[df['send_num'] == df['receive_num'], 'index'].index
df['same'] = 0

for idx in same_idx:
    df.loc[idx, 'same'] = 1

In [40]:
df['gri'] = df['cate_mid'].apply(lambda x: 1 if x=='농산물' else 0)

In [11]:
df.head()

,index,send_num,receive_num,cate_big,cate_mid,target,send_num_3,send_num_4,send_num_5,send_num_6,send_num_7,send_num_8,send_num_9,send_num_10,rec_num_3,rec_num_4,rec_num_5,rec_num_6,rec_num_7,rec_num_8,rec_num_9,rec_num_10,same
0,0,1129000014045300,5011000220046300,패션의류,상의,3.0,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0
1,1,1135000009051200,5011000178037300,생활/건강,반려동물,3.0,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0
2,2,1135000030093100,5011000265091400,패션의류,기타패션의류,9.0,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0
3,3,1154500002014200,5011000315087400,식품,농산물,10.0,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,0
4,4,1165000021008300,5011000177051200,식품,가공식품,3.0,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0


In [41]:
df.drop(['index','send_num','receive_num'], axis=1, inplace=True)

In [ ]:
# df['send_4'] = df['SEND_SPG_INNB'].astype(str).apply(lambda x: x[:4])
# df['rec_4'] = df['REC_SPG_INNB'].astype(str).apply(lambda x: x[:4])

# df["send_4"] = df["send_4"].astype(int)
# df["rec_4"] = df["rec_4"].astype(int)

# fig, ax = plt.subplots(figsize=(10,10))
# sns.heatmap(df.corr(), annot=True, fmt=".4f", cmap="Blues")

## 데이터 전처리

In [42]:
train = df[~df['target'].isna()]
test  = df[df['target'].isna()]

test.drop('target', axis=1, inplace=True)

print(df.shape, train.shape, test.shape)

(36640, 20) (32000, 20) (4640, 19)


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [43]:
train['target'] = train['target'].astype('int64')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [44]:
train.head()

,cate_big,cate_mid,target,send_num_3,send_num_4,send_num_5,send_num_6,send_num_7,send_num_8,send_num_9,send_num_10,rec_num_3,rec_num_4,rec_num_5,rec_num_6,rec_num_7,rec_num_8,rec_num_9,rec_num_10,gri
0,패션의류,상의,3,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0
1,생활/건강,반려동물,3,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0
2,패션의류,기타패션의류,9,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0
3,식품,농산물,10,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,1
4,식품,가공식품,3,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0


In [ ]:
# # scaling
# scaler = StandardScaler()

# scaler.fit(train)

In [45]:
# 원핫 인코딩
train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [46]:
pd.set_option('display.max_rows', None)
train_one.head()

,target,send_num_3,send_num_4,send_num_5,send_num_6,send_num_7,send_num_8,send_num_9,send_num_10,rec_num_3,rec_num_4,rec_num_5,rec_num_6,rec_num_7,rec_num_8,rec_num_9,rec_num_10,gri,cate_big_디지털/가전,cate_big_생활/건강,cate_big_식품,cate_big_여행/문화,cate_big_패션의류,cate_big_화장품/미용,cate_mid_가공식품,cate_mid_건강식품,cate_mid_건강용품,cate_mid_과자,cate_mid_기타디지털/가전,cate_mid_기타식품,cate_mid_기타패션의류,cate_mid_냉동/간편조리식품,cate_mid_농산물,cate_mid_문구/사무용품,cate_mid_문화컨텐츠,cate_mid_반려동물,cate_mid_상의,cate_mid_생활용품,cate_mid_수산,cate_mid_스킨케어,cate_mid_음료,cate_mid_음반,cate_mid_주방용품,cate_mid_축산
0,3,112,1129,11290,112900,1129000,11290000,112900001,1129000014,501,5011,50110,501100,5011000,50110002,501100022,5011000220,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,3,113,1135,11350,113500,1135000,11350000,113500000,1135000009,501,5011,50110,501100,5011000,50110001,501100017,5011000178,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,9,113,1135,11350,113500,1135000,11350000,113500003,1135000030,501,5011,50110,501100,5011000,50110002,501100026,5011000265,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10,115,1154,11545,115450,1154500,11545000,115450000,1154500002,501,5011,50110,501100,5011000,50110003,501100031,5011000315,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,3,116,1165,11650,116500,1165000,11650000,116500002,1165000021,501,5011,50110,501100,5011000,50110001,501100017,5011000177,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## 모델링

In [16]:
!pip install catboost

     |████████████████████████████████| 76.3 MB 1.2 MB/s 


In [47]:
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor 
train_Y = train_one['target']
train_X = train_one.drop('target',axis = 1)


#모델 정의
model = CatBoostRegressor()

In [48]:
# 모델 학습
model.fit(train_X,train_Y)

Learning rate set to 0.070793
0:	learn: 5.7360403	total: 10.4ms	remaining: 10.4s
1:	learn: 5.6913511	total: 29.6ms	remaining: 14.8s
2:	learn: 5.6526941	total: 38.4ms	remaining: 12.8s
3:	learn: 5.6257279	total: 46.8ms	remaining: 11.7s
4:	learn: 5.5963290	total: 55.2ms	remaining: 11s
5:	learn: 5.5660139	total: 63.5ms	remaining: 10.5s
6:	learn: 5.5585967	total: 72.3ms	remaining: 10.3s
7:	learn: 5.5354278	total: 80.8ms	remaining: 10s
8:	learn: 5.5123528	total: 89.3ms	remaining: 9.83s
9:	learn: 5.4912899	total: 97.8ms	remaining: 9.68s
10:	learn: 5.4702969	total: 106ms	remaining: 9.54s
11:	learn: 5.4648410	total: 115ms	remaining: 9.45s
12:	learn: 5.4612564	total: 123ms	remaining: 9.38s
13:	learn: 5.4573965	total: 132ms	remaining: 9.3s
14:	learn: 5.4421103	total: 145ms	remaining: 9.54s
15:	learn: 5.4386611	total: 154ms	remaining: 9.47s
16:	learn: 5.4233258	total: 162ms	remaining: 9.37s
17:	learn: 5.4198066	total: 170ms	remaining: 9.27s
18:	learn: 5.4156863	total: 178ms	remaining: 9.2s
19:	lea

In [49]:
# test 데이터 예측
pred = model.predict(test_one)

In [24]:
pred[:10]

array([ 4.21514683,  4.19551989,  4.46838714,  5.42984391,  4.44597355,
       33.96983114,  4.52651283,  4.65474795,  4.70215345,  4.81784919])

## 정답파일 생성

In [50]:
submission.drop('target', axis=1, inplace=True)

KeyError: ignored

In [51]:
submission['INVC_CONT'] = pred

In [53]:
submission.to_csv('/content/drive/MyDrive/물류예측/baseline_7.csv',index = False)

In [28]:
sub = pd.read_csv('baseline.csv')

In [29]:
sub.head()

,index,INVC_CONT
0,32000,4.215147
1,32001,4.195520
2,32002,4.468387
3,32003,5.429844
4,32004,4.445974
